In [1]:
from stable_baselines3.common.env_checker import check_env
from gym_reversi import ReversiEnv

env = ReversiEnv(player_color='black', opponent = "random", observation_type="numpy3c", 
                illegal_place_mode="lose", board_size=8)
# It will check your custom environment and output additional warnings if needed
check_env(env)


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:30: UserWarning: It seems that your observation  is an image but its `dtype` is (int32) whereas it has to be `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:38: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:51: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


In [3]:
# import sys
# import os
# current_path = os.getcwd()
# sys.path.append(current_path)

from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import PPO
import time
from gym_reversi import ReversiEnv

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# env = make_atari_env("BreakoutNoFrameskip-v4", seed=0)
# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, 
# #                          seed=0
#                         )
# # Frame-stacking with 4 frames
# vec_env = VecFrameStack(vec_env, n_stack=4)

env = ReversiEnv(player_color='black', opponent = "random", observation_type="numpy3c", 
                board_size=8, replace_invalid_action=False, only_game_finished_reward=True)


model = PPO('MlpPolicy', env,
#               policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=2.5e-4,
              ent_coef=0.01,
              n_steps=128,
              n_epochs=4,
              batch_size=256,
              gamma=0.99,
              gae_lambda=0.95,
              clip_range=0.1,
              vf_coef=0.5,
              verbose=1,
              tensorboard_log="models/Reversi_ppo/")

# model = PPO.load("models/Reversi_ppo/model", env=env)

# model.learn(int(2e4))
model.learn(total_timesteps=50_000)
model.save("models/Reversi_ppo/model")

print(f"train time: {time.time()-t0}")

# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
# vec_env = VecFrameStack(vec_env, n_stack=4)

# model = PPO.load("models/Reversi_ppo/model", env=env)
# obs, info = env.reset()
# while True:
#     action, _states = model.predict(obs, deterministic=False)
#     print(f"action: {action}")
#     obs, rewards, dones, truncated, info = env.step(action)
#     env.render("human")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to models/Reversi_ppo/PPO_13


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 128`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 128
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.23     |
|    ep_rew_mean     | -1       |
| time/              |          |
|    fps             | 208      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.13          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 193           |
|    iterations           | 2             |
|    time_elapsed         | 1             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 0.00026457012 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -4.16         |
|    explained_variance   | -2.63         |


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.25        |
|    ep_rew_mean          | -1          |
| time/                   |             |
|    fps                  | 191         |
|    iterations           | 11          |
|    time_elapsed         | 7           |
|    total_timesteps      | 1408        |
| train/                  |             |
|    approx_kl            | 0.000226303 |
|    clip_fraction        | 0           |
|    clip_range           | 0.1         |
|    entropy_loss         | -4.15       |
|    explained_variance   | -27.8       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0466     |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.00279    |
|    value_loss           | 0.00217     |
-----------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.31          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 197           |
|    iterations           | 20            |
|    time_elapsed         | 12            |
|    total_timesteps      | 2560          |
| train/                  |               |
|    approx_kl            | 0.00025647413 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -4.12         |
|    explained_variance   | -0.149        |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0384       |
|    n_updates            | 76            |
|    policy_gradient_loss | -0.00286      |
|    value_loss           | 0.022         |
-------------------------------------------
-------------------------------------------
| rollout/                |     

...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 16            |
|    ep_rew_mean          | -0.98         |
| time/                   |               |
|    fps                  | 216           |
|    iterations           | 347           |
|    time_elapsed         | 205           |
|    total_timesteps      | 44416         |
| train/                  |               |
|    approx_kl            | 0.00033887243 |
|    clip_fraction        | 0.00195       |
|    clip_range           | 0.1           |
|    entropy_loss         | -3.72         |
|    explained_variance   | -1.18         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.037        |
|    n_updates            | 1384          |
|    policy_gradient_loss | -0.00342      |
|    value_loss           | 0.0188        |
-------------------------------------------
------------------------------------------
| rollout/                |      

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 4.36         |
|    ep_rew_mean          | -1           |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 357          |
|    time_elapsed         | 211          |
|    total_timesteps      | 45696        |
| train/                  |              |
|    approx_kl            | 0.0002066209 |
|    clip_fraction        | 0            |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.37        |
|    explained_variance   | -5.51        |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0225      |
|    n_updates            | 1424         |
|    policy_gradient_loss | 9.63e-05     |
|    value_loss           | 0.00301      |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 13.1         |
|    ep_rew_mean          | -1           |
| time/                   |              |
|    fps                  | 216          |
|    iterations           | 366          |
|    time_elapsed         | 216          |
|    total_timesteps      | 46848        |
| train/                  |              |
|    approx_kl            | 4.432304e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.32        |
|    explained_variance   | -1.12        |
|    learning_rate        | 0.00025      |
|    loss                 | 0.0205       |
|    n_updates            | 1460         |
|    policy_gradient_loss | 0.000348     |
|    value_loss           | 0.104        |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | -0.98       |
| time/                   |             |
|    fps                  | 216         |
|    iterations           | 375         |
|    time_elapsed         | 221         |
|    total_timesteps      | 48000       |
| train/                  |             |
|    approx_kl            | 0.009303866 |
|    clip_fraction        | 0.227       |
|    clip_range           | 0.1         |
|    entropy_loss         | -3.77       |
|    explained_variance   | 0.143       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0391     |
|    n_updates            | 1496        |
|    policy_gradient_loss | -0.00267    |
|    value_loss           | 0.00185     |
-----------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 11

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.83          |
|    ep_rew_mean          | -1            |
| time/                   |               |
|    fps                  | 216           |
|    iterations           | 384           |
|    time_elapsed         | 226           |
|    total_timesteps      | 49152         |
| train/                  |               |
|    approx_kl            | 0.00022671791 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | -2.4          |
|    explained_variance   | -0.961        |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0174       |
|    n_updates            | 1532          |
|    policy_gradient_loss | -0.00143      |
|    value_loss           | 0.0214        |
-------------------------------------------
--------------------------------------------
| rollout/                |    

In [3]:
tensorboard --logdir ./reversi/models/Reversi_ppo/PPO_7/ --port=6016

8

In [4]:
25000/3600

6.944444444444445

In [5]:
model = PPO.load("models/Reversi_ppo/model", env=env)

env = ReversiEnv(player_color='black', opponent = "random", observation_type="numpy3c", 
                board_size=8, replace_invalid_action=True, only_game_finished_reward=True)

obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    print(f"action: {action}")
    obs, rewards, dones, truncated, info = env.step(action)
    env.render("human")
    if dones:
        break


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
action: 44
       1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  | 
 -------------------------------------------------------
 1  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 2  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 3  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 4  |  O  |  O  |  O  |  W  |  B  |  O  |  O  |  O  |
 -------------------------------------------------------
 5  |  O  |  O  |  O  |  B  |  W  |  O  |  O  |  O  |
 -------------------------------------------------------
 6  |  O  |  O  |  O  |  O  |  B  |  W  |  O  |  O  |
 -------------------------------------------------------
 7  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |  O  |
 -------------------------------------------------------
 8  |  O  |  O  |  O  |

 2  |  O  |  B  |  O  |  W  |  W  |  B  |  O  |  O  |
 -------------------------------------------------------
 3  |  W  |  W  |  W  |  W  |  W  |  B  |  W  |  O  |
 -------------------------------------------------------
 4  |  O  |  W  |  W  |  B  |  B  |  W  |  W  |  O  |
 -------------------------------------------------------
 5  |  W  |  W  |  B  |  W  |  B  |  B  |  W  |  O  |
 -------------------------------------------------------
 6  |  W  |  W  |  W  |  W  |  B  |  W  |  W  |  O  |
 -------------------------------------------------------
 7  |  W  |  W  |  B  |  B  |  W  |  W  |  W  |  B  |
 -------------------------------------------------------
 8  |  W  |  O  |  B  |  W  |  W  |  W  |  W  |  O  |
 -------------------------------------------------------
action: 52
       1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  | 
 -------------------------------------------------------
 1  |  O  |  O  |  O  |  O  |  W  |  O  |  O  |  O  |
 ---------------------------------------------

1

In [3]:
from gymnasium.utils import seeding


In [4]:
seeding.np_random??

In [5]:
import numpy.random as npr
rng=npr.default_rng()
size=(3,4)
C=rng.uniform(4,7,size)
print(f"{C=}")


C=array([[5.43019184, 6.40250697, 6.1004084 , 5.13867582],
       [6.29431658, 5.80315648, 5.16903573, 4.87083518],
       [4.42687125, 4.45198253, 5.73728282, 6.59846583]])


In [10]:
rng.integers(2)

0

In [ ]:
np.random.Generator()